# TME 6 : Mini-projet Detection de motifs


## Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME5, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faites une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [26]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

def generateRandomSequence(n, I):
    seq_tmp = ""
    
    if I == True:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc)
    else:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc).lower()

    return seq_tmp
    

def generateRandomSequences(n, t):
    sequences = [generateRandomSequence(n, False) for i in range(t)]
   
    return sequences


def implantMotifVar(k, v, t, n):
    sequences = generateRandomSequences(n, t)
    motif = generateRandomSequence(k, True)
    print("Motif : ", motif)
    
    for i in range(len(sequences)):
        L_seq = list(sequences[i])
        L_motif = list(motif)
        for nb_mutation in range(v):
            index_mutation_motif = random.randint(0,len(motif) - 1)
            L_motif[index_mutation_motif] = random.choice(nuc)
        chn_motif_mute = "".join(L_motif)
        print(chn_motif_mute)
        L_seq.insert(random.randint(0, len(L_seq) - 1), chn_motif_mute)
        
        sequences[i] = "".join(L_seq)
    return sequences

sequences = implantMotifVar(k, v, t, n)
print ("ADN : ", sequences)


Motif :  GATT
GATT
GTTT
GCTT
ADN :  ['attagagttGATTc', 'agaagataaGTTTc', 'gGCTTtagcccggc']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [27]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

sequences = implantMotifVar(k, v, t, n)
print (sequences)

Motif :  AATGCTTC
CATGCTTC
CATGCCTC
TATGCTTC
TCTGCTTC
AATGCGTC
AATGCTGC
AGTGCTTC
CATGCTTC
CATGCTTC
AACGCTTG
['agacgtctcCATGCTTCt', 'tcCATGCCTCactctatg', 'cTATGCTTCacatcctgc', 'tagagaTCTGCTTCttaa', 'ggtaagAATGCGTCatcg', 'tAATGCTGCaacacttac', 'ttgAGTGCTTCagttttg', 'ggtggtcCATGCTTCgca', 'tcaCATGCTTCaatagaa', 'atggaaAACGCTTGagac']


3\. Préparation pour l'algorithme _"Brute Force Motif Search"_. Avant d'implementer  l'algorithme _"Brute Force Motif Search"_, faites une fonction pour calculer le score `score(s, seqs)`, où `s` est un ensemble de position initiales `s`$=(s_1,s_2,...,s_k)$  et `seqs` est une matrice de taille `t x n`  (`t` est le nombre de séquences et `n` la taille de ces séquences), pour tester votre fonction utiliser la matrice et les valeurs ci-dessous (voir aussi les slides du cours).


In [82]:
import numpy as np

k =8
n = 68 # length of each DNA sequence
t = 5 # number of  DNA sequence samples

s=(25,20,2,55,59)

seqs = [
'CCTGATAGACGCTATCTGGCTATCCAGGTACTTAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATCCATACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAACGTTAGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTCCATATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTACCGTACGGC'
]



#extract sequences from seqs
def extractSeqs(s, seqs):
    motifs = []
    
    L_zip = list(zip(s,seqs))
    
    for i_ini, seq in L_zip:
        seq_tmp = ""
        for i in range(i_ini, i_ini + k):
            seq_tmp = seq_tmp + seq[i]
        motifs.append(seq_tmp)
                
    return motifs


#construct Profiles
def profile(motifs, k, t, q):
    PWM = np.zeros((q, k), dtype = int)
    L = [x for x in nuc]
    
    dictionnaire = {}
    for i in range(len(L)):
        dictionnaire[L[i]] = PWM[i]
    
    for i in range(k):
        for j in range(len(motifs)):
            dictionnaire[motifs[j][i]][i] = dictionnaire[motifs[j][i]][i] + 1
            
    
    for i in range(q):
        PWM[i] = dictionnaire[L[i]]
        
    print(PWM)
    
    return PWM


def score(s, seqs):
    sc = 0
    profile1 = profile(extractSeqs(s, seqs), k, t, len(nuc))
    
    L_max_colonne = []
    
    for i in range(len(profile1[0])):
        L_tmp = []
        for j in range(len(nuc)):
            L_tmp.append(profile1[j][i])
        sc = sc + max(L_tmp)
    
    return (sc)

sc = score(s, seqs)
#teste, sc doit etre egalle à 30
print (sc)



[[3 0 1 0 3 1 1 0]
 [2 4 0 0 1 4 0 0]
 [0 1 4 0 0 0 3 1]
 [0 0 0 5 1 0 1 4]]
30


4\. Implementer l'algoritme _"Brute Force Motif Search"_ pour chercher des motifs de taille variable. Attention, la complexité de cet algorithme est trop importante, car il enumère toutes les positions intiales possibles, nous allons donc le tester seulement sur de petits jeux de données.

In [ ]:
from itertools import product


#enumeration all initial positions
def initialPosition(k, n, t):
	iniPos = []; allIniPos = [];
	for i in range(0, n-k+1):
		iniPos.append(i)
        
	for subset in product(iniPos, repeat=t):
		allIniPos.append(subset)
	return allIniPos

def bruteForceMotifSearch(sequences, t, n, k):
	bestscore = 0; bestMotif=''

	return bestMotif, bestscore

#test
k=4; v=1; t=3; n=10
sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)
print (bestMotif, bestscore)

5\. L'algorithme est-il performant sur ce jeu de données ? Sinon, pourquoi ?

<font color='blue'>
réponse: 
</font>

6\. Tester l'agorithme avec les paramètres suivants `k=7`, `v=1`, `t=5`, et `n=20`. Avez vous de meilleurs performances ?

In [ ]:
k=7; v=1; t=5; n=20

sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)

print (bestMotif)

7\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les mêmes jeux de données que la question 3 (voir slides de cours). Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles.

In [ ]:
#generer les k-mers valid
validkmers = []


In [ ]:

def hamdist(v, x):
	diffs = 0

	return diffs


def totalDistance(motif, sequences, k):
	td = 0
	
	return td


def MedianStringSearch(allkmers, sequences, t, n, k):
	bestDistance = 1000
	bestMotif = ""
	motifDist = {} #all motifs found

	return bestMotif, bestDistance, motifDist

#teste, td = 4
td = totalDistance("acgtacgt", sequences, k)
print (td)
bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, sequences, t, n, k)
print (bestMotif, bestDistance)

8\. Comparation entre les algorithmes _"Median String Search"_ et "_Brute Force Motif Search_"

a\. Produire des séquences artificielles avec les paramètres suivants  `k=7`, `v=1`, `t=5`, `n=20`. Comparer les performance et le temps de calculs des deux algorithmes.

In [ ]:
k=7; v=1; t=5; n=20

b\. Quel algorithme peut etre appliqué au probleme de recherche de motifs? Et comment?

<font color='blue'>
réponse: 
</font>

In [ ]:
k=7; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    return sequence

genome = ""


sequences   = readFasta(genome, n)

t = len(sequences)


9\. Ulitser la base YEASTRACT : http://www.yeastract.com/formsearchbydnamotif.php pour chercher les motifs. 
Avez vous une indication pour le facteur de transcription impliqué ? Comparer vos résultats aux résultats obtenu avec algorithme Table de Hachage.


<font color='blue'>
réponse: 
</font>